In [3]:
#1. get xmlboxes
#2. get xmlboxes img region from bkg
#3. put the img region to new image
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
generated_image_dir = '/data/darknet/python/' + time_str + '_images_replaced_ordinary_obj_with_bkg/'

time_str = '20190123'
xml_dir = '/data/darknet/python/' + time_str + '_xml/'
image_dir = '/data/darknet/python/' + time_str + '_image/'

os.system('mkdir -p ' + xml_dir)
os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
background_dir = '/data/darknet/python/background/'

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
SEED_BKG_DIR = background_dir
GENE_IMG_DIR = generated_image_dir


In [4]:
def past_to_background_from_image_file(file, bboxes, background_img_array, save_name, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    time_mark = datetime.datetime.now()
    time_str = time_mark.strftime("%Y%m%d%H%M%S.%f")
    cv2.imwrite(GENE_IMG_DIR + time_str + '_' + save_name.split('/')[-1], background_img_array)
    return 
def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        '''
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        '''
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, tree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_bkg_names = os.listdir(SEED_BKG_DIR)
seed_bkg_names.sort()
seed_xml_names.sort()

for xml_name in seed_xml_names:
    print(xml_name)       
    if(not ((int(xml_name[:-4]) >= 460500) and int(xml_name[:-4]) < 461000)):
        continue
    background_img_data = cv2.imread(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    if(not xml_name.endswith('.xml')):
        continue
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    bboxes = get_bboxes_from_etree(tree)
    past_to_background_from_image_file(SEED_BKG_DIR + seed_bkg_names[10], bboxes, background_img_data, 
                                       SEED_IMG_DIR + xml_name[:-3] + 'jpg')

00000460000.xml
00000460001.xml
00000460002.xml
00000460003.xml
00000460004.xml
00000460005.xml
00000460006.xml
00000460007.xml
00000460008.xml
00000460009.xml
00000460010.xml
00000460011.xml
00000460012.xml
00000460013.xml
00000460014.xml
00000460015.xml
00000460016.xml
00000460017.xml
00000460018.xml
00000460019.xml
00000460020.xml
00000460021.xml
00000460022.xml
00000460023.xml
00000460024.xml
00000460025.xml
00000460026.xml
00000460027.xml
00000460028.xml
00000460029.xml
00000460030.xml
00000460031.xml
00000460032.xml
00000460033.xml
00000460034.xml
00000460035.xml
00000460036.xml
00000460037.xml
00000460038.xml
00000460039.xml
00000460040.xml
00000460041.xml
00000460042.xml
00000460043.xml
00000460044.xml
00000460045.xml
00000460046.xml
00000460047.xml
00000460048.xml
00000460049.xml
00000460050.xml
00000460051.xml
00000460052.xml
00000460053.xml
00000460054.xml
00000460055.xml
00000460056.xml
00000460057.xml
00000460058.xml
00000460059.xml
00000460060.xml
00000460061.xml
00000460

00000460513.xml
00000460514.xml
00000460515.xml
00000460516.xml
00000460517.xml
00000460518.xml
00000460519.xml
00000460520.xml
00000460521.xml
00000460522.xml
00000460523.xml
00000460524.xml
00000460525.xml
00000460526.xml
00000460527.xml
00000460528.xml
00000460529.xml
00000460530.xml
00000460531.xml
00000460532.xml
00000460533.xml
00000460534.xml
00000460535.xml
00000460536.xml
00000460537.xml
00000460538.xml
00000460539.xml
00000460540.xml
00000460541.xml
00000460542.xml
00000460543.xml
00000460544.xml
00000460545.xml
00000460546.xml
00000460547.xml
00000460548.xml
00000460549.xml
00000460550.xml
00000460551.xml
00000460552.xml
00000460553.xml
00000460554.xml
00000460555.xml
00000460556.xml
00000460557.xml
00000460558.xml
00000460559.xml
00000460560.xml
00000460561.xml
00000460562.xml
00000460563.xml
00000460564.xml
00000460565.xml
00000460566.xml
00000460567.xml
00000460568.xml
00000460569.xml
00000460570.xml
00000460571.xml
00000460572.xml
00000460573.xml
00000460574.xml
00000460